In [192]:
import cvxpy as cp
import numpy as np

In [193]:
DISCOUNT_FACTOR = 0.95

DELIVER_PROBS = np.array([
    [0.75, 0.075, 0.1, 0.075],
    [0.15, 0.7, 0.06, 0.09],
    [0.14, 0.105, 0.65, 0.105],
    [0.05, 0.025, 0.025, 0.9]
])

WAIT_PROBS = np.identity(4)

TRAVELLING_COST = np.array([
    [0, 1.5, 2.0, 2.5],
    [1.0, 0, 1.8, 2.2],
    [1.8, 1.5, 0, 1.2],
    [2.0, 1.8, 1.0, 0]
])

SOURCE_DESC_WITH_PACKAGE_FEE = np.array([
    [0, 12, 8, 6],
    [10, 0, 7, 9],
    [14, 9, 0, 5],
    [15, 7, 4, 0]
])

reward = SOURCE_DESC_WITH_PACKAGE_FEE  - TRAVELLING_COST

V = cp.Variable(4)

# Objective is to minimize the sum over V values
objective =cp.Minimize(cp.sum(V))

# Building the Constraints
constraints_wait_action = []
constraints_deliver_action = []
for s in range(4):

    temp_wait = 0
    temp_deliver = 0

    for s_prime in range(4):
        #For wait action reward is zero
        temp_wait += WAIT_PROBS[s, s_prime]*(0.0 + DISCOUNT_FACTOR*V[s_prime])        
        temp_deliver += DELIVER_PROBS[s, s_prime]*(reward[s, s_prime] + DISCOUNT_FACTOR*V[s_prime])
    
    constraints_wait_action.append(V[s]>=temp_wait)
    constraints_deliver_action.append(V[s]>=temp_deliver)

constraints = constraints_wait_action + constraints_deliver_action

prob = cp.Problem(objective, constraints)
result = prob.solve()
V.value

array([32.45984984, 33.60654623, 34.85305452, 27.74316124])